In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import scanpy as sc
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns

In [ ]:
# adata = sc.read_h5ad("/storage/codex/datasets_v1/bladder_merged_v5.h5ad")
# adata

ad = sc.read_h5ad("/storage/codex/preprocessed_data/210309_Final_BoneMarrow_reg0/210309_Final_BoneMarrow_reg0.h5ad")
ad

In [ ]:
# ad = adata[adata.obs.sample_id=='210226_Bladder_TMA1_reg10_v4']
# ad

In [ ]:
# https://docs.bokeh.org/en/latest/docs/gallery/color_sliders.html
def hex_to_dec(hex):
    red = ''.join(hex.strip('#')[0:2])
    green = ''.join(hex.strip('#')[2:4])
    blue = ''.join(hex.strip('#')[4:6])
    return np.array([int(red, 16), int(green, 16), int(blue,16)])/255


def make_spatial_distributions(coords, celltypes, N=100, overlap=0.1, mode='distribution'):
    """
    modes
    distribution: distribution of cell types at postion X
    relative-abundance: relative abundance of cell type at position X compared
        to the rest of the spatial axis. `stackplot` parameter `baseline` can be
        be `sym` or `wiggle`
    total: raw counts
    """
    #colors = [hex_to_dec(d) for d in ad.uns['celltype_gating_colors']]
    spatial_coord = coords[:,0].copy()

    u_cells = np.unique(celltypes)
    x = np.linspace(np.min(coords[:,0]), np.max(coords[:,0]), N+1)
    y = np.zeros((len(u_cells), N))
    start = np.min(coords[:,0])
    step = x[1]-x[0]
    ovl = step * overlap
    for i in range(N):
        s = x[i]-ovl
        e = x[i+1]+ovl
        ix = (spatial_coord>s) & (spatial_coord<e)
        u, cnt = np.unique(celltypes[ix], return_counts=True)
        p = np.zeros(len(u_cells))
        for k,c in enumerate(cnt):
            p[u_cells==u[k]] = c
        y[:,i] = p.copy()
        
    if mode=='distribution': 
        y = y / np.sum(y,axis=0)
    elif mode=='relative-abundance':
        y = (y.T / np.max(y,axis=1)).T
    elif mode=='total':
        pass

    return np.arange(N), y

In [ ]:
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes

# s = np.unique(adata.obs.sample_id)
# s_ = np.random.choice(s)
# ad = adata[adata.obs.sample_id==s_]

# theta = np.random.choice(180)
# rot = np.array([[np.cos(theta), -np.sin(theta)],
#                 [np.sin(theta), np.cos(theta)]])

coords = ad.obsm['coordinates_t'].copy()
celltypes = np.array(ad.obs.celltype_gating)
u_cells = np.unique(celltypes)

colors = sns.color_palette('tab20', len(np.unique(celltypes)))

rcParams['figure.dpi'] = 300
r = np.max(np.abs(coords), axis=0)
r = r[0]/r[1]
fig, axs = plt.subplots(2,1, figsize=(3*r,4), 
                        gridspec_kw=dict(
                            height_ratios=(5,1), 
                            hspace=0.1, )
                       )
ax = axs[0]
_ , celltypes_numeric = np.unique(celltypes, return_inverse=True)
color_list = [colors[i] for i in celltypes_numeric]
ax.scatter(coords[:,0], coords[:,1], s=1, color=color_list)

# for i,j in enumerate(u_cells):
#     ix = celltypes==j
#     c = coords[ix]
#     ax.scatter(c[:,0], c[:,1], label=j, s=1, color=colors[i])
    
ax.legend(bbox_to_anchor=(1,1),markerscale=7, frameon=False)
_ = ax.set_xticks([])
_ = ax.set_yticks([])

ax = axs[1]
x, y = make_spatial_distributions(coords, celltypes, N=50, mode='distribution')
_ = ax.stackplot(x, y, baseline='zero', colors=colors)
_ = ax.set_xticks([])



In [ ]:
i